In [57]:
import pandas as pd
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error

In [7]:
LCL_wks = pd.read_pickle('data/LCL_unstack_wks.pkl')

In [8]:
LCL_wks.index

DatetimeIndex(['2012-01-01', '2012-01-08', '2012-01-15', '2012-01-22',
               '2012-01-29', '2012-02-05', '2012-02-12', '2012-02-19',
               '2012-02-26', '2012-03-04',
               ...
               '2013-12-29', '2014-01-05', '2014-01-12', '2014-01-19',
               '2014-01-26', '2014-02-02', '2014-02-09', '2014-02-16',
               '2014-02-23', '2014-03-02'],
              dtype='datetime64[ns]', name='DateTime', length=114, freq='W-SUN')

In [31]:
ToU_test = LCL_wks['ToU'].tail(round(0.09*LCL_wks["ToU"].count()))
ToU_train = LCL_wks['ToU'].loc[LCL_wks.index < ToU_test.index[0]]

In [53]:
ES = ExponentialSmoothing(endog = ToU_train, seasonal = 'mul', trend = 'add', seasonal_periods=52).fit()

all_pred = ES.predict(start=LCL_wks['ToU'].index[0], end = LCL_wks['ToU'].index[LCL_wks.shape[0]-1])
pred_train = ES.predict(ToU_train.index[0], end=ToU_train.index[len(ToU_train)-1])
pred_test = ES.predict(ToU_test.index[0], end=ToU_test.index[len(ToU_test)-1])

In [54]:
fig = go.Figure()
fig.add_trace(go.Line(x=ToU_train.index, y=ToU_train, mode='lines', name="Train"))
fig.add_trace(go.Line(x=ToU_test.index, y=ToU_test, mode='lines', name="Test"))
# fig.add_trace(go.Line(x=all_pred.index, y=all_pred, mode='lines', name="Predictions"))
fig.add_trace(go.Line(x=ToU_test.index, y=pred_test, mode='lines', name="Test Predictions"))
fig.add_trace(go.Line(x=ToU_train.index, y=pred_train, mode='lines', name="Train Predictions"))
fig.show()

c:\Users\triha\anaconda3\envs\timeseries\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [58]:
mean_absolute_percentage_error(ToU_test,pred_test)

0.03531693385830563